In [2]:
import json,os,re
import numpy as np
import pandas as pd

In [37]:
path = './data/comment_classify2/train_data.txt'
with open(path,'r',encoding='utf-8') as fp:
    data = fp.readlines()

all_data = []
for sample in data:
    all_data.append(json.loads(sample))

In [38]:
support_list = []
query_list = []
for sample in all_data:
    category_name = sample['category_name']
    category_description = sample['category_description'].replace('\t','##')
    support_set = sample['support_set']
    query_set = sample['query_set']
    for support in support_set:
        text = support.get('text').replace('\t','##')
        support_list.append(f"{category_name}：{category_description}\t{text}\t{support.get('label')}")

    len_add = int(0.7*len(query_set))
    for query in query_set[:len_add]:
        text = query.get('text').replace('\t','##')
        support_list.append(f"{category_name}：{category_description}\t{text}\t{query.get('label')}")

    for query in query_set[len_add:]:
        text = query.get('text').replace('\t','##')
        query_list.append(f"{category_name}：{category_description}\t{text}\t{query.get('label')}")

print("补充后................................")
print('len_support_list：',len(support_list))
print('len_query_list：',len(query_list))
print("测试集比例：",len(query_list) /(len(query_list) + len(support_list)))

补充后................................
len_support_list： 75000
len_query_list： 30000
测试集比例： 0.2857142857142857


In [39]:
with open('./data/comment_classify2/train.txt','w',encoding='utf-8') as fp:
    for sample in support_list:
        fp.write(sample.strip() + '\n')

with open('./data/comment_classify2/dev.txt','w',encoding='utf-8') as fp:
    for sample in query_list:
        fp.write(sample.strip() + '\n')

In [14]:
import pandas as pd
import random
import json,os,re
import numpy as np

In [15]:
path = './data/comment_classify2/train_data.txt'
with open(path,'r',encoding='utf-8') as fp:
    data = fp.readlines()
all_data = []
for sample in data:
    all_data.append(json.loads(sample))

support_list = []
query_list = []
for sample in all_data:
    category_name = sample['category_name']
    category_description = sample['category_description'].replace('\t','##')
    support_set = sample['support_set']
    query_set = sample['query_set']
    for support in support_set:
        text = support.get('text').replace('\t','##')
        support_list.append(f"{category_name}：{category_description}\t{text}\t{support.get('label')}")

    len_add = int(0.7*len(query_set))
    for query in query_set[:len_add]:
        text = query.get('text').replace('\t','##')
        support_list.append(f"{category_name}：{category_description}\t{text}\t{query.get('label')}")

    for query in query_set[len_add:]:
        text = query.get('text').replace('\t','##')
        query_list.append(f"{category_name}：{category_description}\t{text}\t{query.get('label')}")

In [16]:
path = './data/comment_classify2/test_data.txt'
with open(path,'r',encoding='utf-8') as fp:
    data = fp.readlines()
all_data = []
for sample in data:
    all_data.append(json.loads(sample))

task_id_list = []
record_id_list = []
category_description_list = []
text_list = []


for sample in all_data:
    false_sample_list = []
    task_id = sample['task_id']
    category_name = sample['category_name']
    category_description = sample['category_description'].replace('\t','##')
    support_set = sample['support_set']
    query_set = sample['query_set']
    for support in support_set[:3]:
        text = support.get('text').replace('\t','##')
        support_list.append(f"{category_name}：{category_description}\t{text}\t{support.get('label')}")

    for support in support_set[3:]:
        text = support.get('text').replace('\t','##')
        query_list.append(f"{category_name}：{category_description}\t{text}\t{support.get('label')}")
    # 进行负采样
    false_sample_list = [f"{category_name}：{category_description}\t"+'\t'.join(sample.split('\t')[1:])[:-1] + '0' for sample in random.sample(support_list,1000) if sample.split('：')[0] != category_name][:80]
    if len(false_sample_list) <80:
        s = 1 / 0
    support_list += false_sample_list[:50]
    query_list += false_sample_list[50:]

    # 测试集
    for query in query_set:
        text = query.get('text').replace('\t','##')
        record_id = query.get('record_id')
        task_id_list.append(task_id)
        record_id_list.append(record_id)
        category_description_list.append(f"{category_name}：{category_description}")
        text_list.append(text)

In [17]:
false_sample_list

['车辆违停：该类数据一般为市民等发布的关于某地域存在车辆违停问题的举报、投诉、曝光等信息\t【标题】：#我的抖音生活日记【正文】：#我的抖音生活日记【封面_OCR】：125 三月十五 给您！ 【抽帧_OCR】：美好祝愿 天好心情 幸福安康 1月25日 送给您 送给您 农历十二月十五星期四 J25【语音转写】：早上好朋友们，新的一天开始。\t0',
 '车辆违停：该类数据一般为市民等发布的关于某地域存在车辆违停问题的举报、投诉、曝光等信息\t【标题】：【正文】：【封面_OCR】：【抽帧_OCR】：【语音转写】：又到年底了，辛苦了一年的农民工朋友们，你们的工资收全了吗？收到了吗？欠薪的情况连绵不绝，每年都有，如果收不到工资，下面的方法你一定要记好了，因为这个方法是从上面。\t0',
 '车辆违停：该类数据一般为市民等发布的关于某地域存在车辆违停问题的举报、投诉、曝光等信息\t【标题】：这波是努力局#锄禾日当午 #花一万学的街舞【正文】：这波是努力局#锄禾日当午 #花一万学的街舞【封面_OCR】：【抽帧_OCR】：【语音转写】：\t0',
 '车辆违停：该类数据一般为市民等发布的关于某地域存在车辆违停问题的举报、投诉、曝光等信息\t【标题】：废弃医院惊现诡异护士！#鬼故事 #灵异 #灵异故事 #探秘 #恐怖悬疑【正文】：废弃医院惊现诡异护士！#鬼故事 #灵异 #灵异故事 #探秘 #恐怖悬疑【封面_OCR】：论异医院护士 楚废弃医院来的论异护生畸 【抽帧_OCR】：她的眼神变得柔和了许多 C 他们开始四处寻找逃脱的方法 恐惧笼罩着每一个人 门窗紧闭仿佛被一股无形的力量所控制 100 TROPICANA 她始终没有开口说过一句话 随着时间的推移 G 但她似乎并不理会他们的话语 王 这个你得问阎王 无论他们如何尝试与护士沟通 探险者们不禁感到一阵寒意 RN·IIFT8 她的眼睛直勾勾地盯着他们 脸上浮现出一个诡异的微笑 她正静静地站在手术台旁 18:45 请保持安静 外科楼三楼 走廊两侧的病房门吱吱作响 揭开护士的秘密 中士站 他们意识到自己已经被困在了这座废弃医院中 面无表情眼睛里透露出一种说不出的诡异 她的身影飘忽不定 nrD1 NICAFEE nr1 HICASFE 曾看到一位身着护士服的诡异女子在医院内游荡 TOKYO BELLINI CA HOTEL KENT ar

In [18]:
print("补充后................................")
print('len_support_list：',len(support_list))
print('len_query_list：',len(query_list))
print("测试集比例：",len(query_list) /(len(query_list) + len(support_list)))

补充后................................
len_support_list： 101500
len_query_list： 46000
测试集比例： 0.31186440677966104


In [19]:
with open('./data/comment_classify2/train.txt','w',encoding='utf-8') as fp:
    for sample in support_list:
        fp.write(sample.strip() + '\n')

with open('./data/comment_classify2/dev.txt','w',encoding='utf-8') as fp:
    for sample in query_list:
        fp.write(sample.strip() + '\n')

df = pd.DataFrame({'task_id':task_id_list,'record_id':record_id_list,'description':category_description_list,'text':text_list})
df.to_csv('./data/comment_classify2/test.csv',index=0,encoding='utf-8-sig')

121